In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from natsort import natsorted
import ast

# Update

In [2]:
strategy_version_input=751
attribute_id_input=4634

# Update

In [3]:
# parameters
customer_id = '5'
formatted_attribute = 'capacity_l'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

In [4]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [5]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [6]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]   

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    tuples=tuple(split_digits(s, *args, **kwargs))
    print(tuples)
    return tuples 

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]



In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
print(len(df))

86


In [8]:
regex_pattern = r"(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}weighi?n?g?\s?\d+\s?lb)|(support.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|(\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?weight capacity)|(\s?\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?.{0,15}weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|()"               
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [9]:
regex_lbs=r"(?i)(\d+\.?\d*\s?lb)|()"
df['newest_matches']=df['matches'].apply(lambda x: re_extract(regex_lbs, str(x)))

df_newest=df[df['newest_matches'].astype(str)!='[]']
df_newest['newest_matches']=df_newest['newest_matches'].apply(lambda x: re.sub(r"(?i) lb",'lb',str(x))).apply(lambda x: re.sub(r'\.lb','lb',str(x))).apply(lambda x: re.sub(r'\.lb','lb',str(x))).apply(lambda x: re.sub(r'lb',' lb',str(x))).apply(lambda x: re.sub(r'  ',' ',str(x)))                 
df_newest['newest_matches']=df_newest['newest_matches'].apply(lambda x: re.sub(r'\[','["',str(x))).apply(lambda x: re.sub(r'\]','"]',str(x))).apply(lambda x: re.sub(r',','","',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r"'",'',str(x)))

In [10]:
print(len(df_newest))
# df_newest[0:5]

0


In [11]:
df_newest[curation_col]=df_newest['newest_matches']
matchesnewest=df_newest[['external_id',curation_col]]

# Na

In [12]:
regex_pattern_na=r'(?i)(up.?to.?\d+\s?lb)|(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}weighi?n?g?\s?\d+\s?lb)|(support.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|(\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?weight capacity)|(\s?\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?.{0,15}weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(Mini?m?u?m?.{0,15}\s?w?e?i?g?h?t?.?\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb)'   
df['matches_na'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

# regex_lb=r'(?i)(?<!weighs\s?)\d+.?\d*\s?lb'
# df['matches_lb'] = df['long_desc'].apply(lambda x: re_extract(regex_lb, x))

df_na=df[(df['matches_na'].astype(str)=='[]')]#&(df['matches_lb'].astype(str)!='[]')] 
df_na[curation_col]='n/a'
match_na=df_na[['external_id',curation_col]]
match_na

external_id Q:capacity_l
1     200001662          n/a
2      69775172          n/a
3      69725004          n/a
4      69725727          n/a
5      69725605          n/a
6      68022280          n/a
7      69781820          n/a
8      69784172          n/a
9      69784172          n/a
10     69784173          n/a
11     69784173          n/a
12     66143680          n/a
13     69725647          n/a
14     41612002          n/a
15    200000161          n/a
16    200001661          n/a
17     69771758          n/a
18     69719142          n/a
19     69624728          n/a
20     69736217          n/a
21     68473266          n/a
22     14241493          n/a
23    200001643          n/a
24     69783668          n/a
25     69767000          n/a
26    200000011          n/a
27     69719126          n/a
28     69792238          n/a
30     69764509          n/a
31     69792065          n/a
32     69771307          n/a
33     69724999          n/a
34     43316151          n/a
35     69705116          n/a
36     69771302          n/a
37     69725728          n/a
39     69776800          n/a
41     69747594          n/a
43     69787002          n/a
45    200001663          n/a
47     69736212          n/a
48     69596805          n/a
49     69705068          n/a
50     69714235          n/a
51     69775162          n/a
52     69725753          n/a
53     69725710          n/a
55     69194795          n/a
57     44926540          n/a
58     69736221          n/a
59     66253815          n/a
61     69779776          n/a
62     69649389          n/a
64    200001660          n/a
65    200001664          n/a
66     44821043          n/a
67     63903638          n/a
68     69724936          n/a
69     69736218          n/a
70     69725796          n/a
71    200001615          n/a
72     69751344          n/a
73     69759812          n/a
74    200001634          n/a
76     69766995          n/a
77     69725750          n/a
78     69725709          n/a
79     69725781          n/a
80     69781821          n/a
81     69736220          n/a
83     69711861          n/a
85     69598161          n/a
86    200005198          n/a
87     69596804          n/a
88     69792236          n/a
89     69720918          n/a
90     69747012          n/a
91     47103047          n/a
93     69719141          n/a
94     69713447          n/a
96     68535094          n/a
97     69724986          n/a
98     69723717          n/a
99    200001667          n/a
100   200001650          n/a
101    69792239          n/a

In [13]:
print(len(df_na))
# df_na[0:500]

86


In [14]:
print(len(df))
print(len(matchesnewest)+len(match_na))

86
86


In [15]:
error

NameError: name 'error' is not defined

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, matchesnewest)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute, df, match_na)